### Dandelion Optimizer

#### Important Links:
- [Paper](https://www.sciencedirect.com/science/article/abs/pii/S0952197622002305)<br>
- [Matlab](https://www.mathworks.com/matlabcentral/fileexchange/114680-dandelion-optimizer)

In [1]:
function initialization(Popsize, Dim, UB, LB)
    Boundary_no = length(UB) # number of boundaries
    
    # Preallocate array for Positions
    Positions = zeros(Popsize, Dim)

    # If the boundaries of all variables are equal and user enters a single number for both UB and LB
    if Boundary_no == 1
        Positions = rand(Popsize, Dim) .* (UB - LB) .+ LB
    end

    # If each variable has a different LB and UB
    if Boundary_no > 1
        for i in 1:Dim
            ub_i = UB[i]
            lb_i = LB[i]
            Positions[:, i] = rand(Popsize) .* (ub_i - lb_i) .+ lb_i
        end
    end

    return Positions
end

initialization (generic function with 1 method)

The above function initializes the starting positions of Dandelions.<br><br>
__Parameters__:
- Popsize: Determines the number of dandelions initialized
- Dim: States the number of dimensions of the problem
- UB, LB: This is an array giving the upper and lower bounds for the initialization values<br>

__Returns__:
- Initial Positions of all the dandelions